# Text Metodları
Genellikle metin verilerinin işlenmesinde temizlenmesi veya manipüle edilmesi gerekir. Örneğin, bir grup e-posta adresinden oluşan bir sütununuz olabilir, böylece gmail.com'de birinin adı gibi birşeye sahip olursunuz. Ve ortak bir görev olabilecek şey, alan adlarını (domain names) e-posta adresindeki gerçek addan ayırmaya çalışmaktır.

## String metodları
Normal bir Python stringi, kullanılabilen çeşitli yöntem çağrılarına sahiptir:

In [1]:
email = 'huzeyfebostan99@gmail.com'

In [2]:
# tab ile string ile kullanılabilen birçok yönteme ulaşabilirsiniz.
email.split('@')

['huzeyfebostan99', 'gmail.com']

In [3]:
email.find('@')

15

In [5]:
email[15]

'@'

In [7]:
email.isdigit()

False

In [8]:
'5'.isdigit()

True

In [11]:
#help(str)

# Pandas ve Text

Pandas burada gösterdiğimizden çok daha fazlasını yapabilir.

## Pandas string sütununda text metodları
Pandas series'teki bir sütunda string yöntemlerini kullanmak için series isminden sonra .str eki ekleyip bu yöntemleri tab tuşuna basıp görülebilir. Python string yöntem çağrılarının tümünü bu şekilde pandas 'ta da çağırılabilir.

In [12]:
import pandas as pd

In [14]:
names = pd.Series(['andrew','bobo','claire','david','4'])

In [15]:
names

0    andrew
1      bobo
2    claire
3     david
4         4
dtype: object

In [16]:
# names series'ın sütunundaki tüm stringler için bu yöntemi uygular.
names.str.capitalize()

0    Andrew
1      Bobo
2    Claire
3     David
4         4
dtype: object

In [17]:
names.str.isdigit()

0    False
1    False
2    False
3    False
4     True
dtype: bool

## Bölme, Alma ve Genişletme

In [19]:
tech_finance = ['GOOG,APPL,AMZN','JPM,BAC,GS'] # bir liste alalım.

In [20]:
len(tech_finance)

2

In [21]:
tickers = pd.Series(tech_finance)

In [22]:
tickers

0    GOOG,APPL,AMZN
1        JPM,BAC,GS
dtype: object

In [23]:
tickers.str.split(',')

0    [GOOG, APPL, AMZN]
1        [JPM, BAC, GS]
dtype: object

In [24]:
tickers.str.split(',').str[0] # pandas'a uygun string alma şekli

0    GOOG
1     JPM
dtype: object

In [25]:
# series'deki üç sütuna bölrek tabloyu genişletebiiriz.
tickers.str.split(',',expand=True)

,0,1,2
0,GOOG,APPL,AMZN
1,JPM,BAC,GS


## Strings Silme veya Düzenleme

In [26]:
messy_names = pd.Series(["andrew  ","bo;bo","  claire  "])

In [27]:
# "andrew  " ve "  claire  " deki boşluk nedeniyle sağ taraftaki 
# 'yanlış hizalamaya' dikkat edin
messy_names

0      andrew  
1         bo;bo
2      claire  
dtype: object

İlk göze çarpan şey bo;bo stringinin içerisindeki ; olduğudur. .str 'den sonra .replace() yöntemini çağırarak bunu ve "" ile yerdeğiştiririz.

In [29]:
messy_names.str.replace(";","")

0      andrew  
1          bobo
2      claire  
dtype: object

In [30]:
messy_names.str.strip() # default olarak stringlerdeki boşlukları siler

0    andrew
1     bo;bo
2    claire
dtype: object

Şimdi ikisini birlikte uygulayarak ; ve gereksiz boşlukları pandas tablosundan aşağıdaki tek kod ile kaldıralım. Her durumda .str çağrısı kullnıldığına dikkat edin.

In [31]:
messy_names.str.replace(";","").str.strip() 

0    andrew
1      bobo
2    claire
dtype: object

In [32]:
# son olarak tüm strinlerin ilk harflerini büyüttük
messy_names.str.replace(";","").str.strip().str.capitalize()

0    Andrew
1      Bobo
2    Claire
dtype: object

##  Özel apply() ile alternatif çağrı
Buraya kadar yapılanları bir fonksiyon olarak tanımlayıp bunu .apply() yöntemi ile pandas series'a uygulayabiliriz.

In [33]:
def cleanup(name):
    name = name.replace(";","")
    name = name.strip()
    name = name.capitalize()
    return name

In [34]:
messy_names

0      andrew  
1         bo;bo
2      claire  
dtype: object

In [35]:
messy_names.apply(cleanup)

0    Andrew
1      Bobo
2    Claire
dtype: object

In [36]:
import numpy as np
pd.Series(np.vectorize(cleanup)(messy_names))

0    Andrew
1      Bobo
2    Claire
dtype: object

## Hangisi daha etkili?

In [37]:
import timeit 
  
# code snippet to be executed only once 
setup = '''
import pandas as pd
import numpy as np
messy_names = pd.Series(["andrew  ","bo;bo","  claire  "])
def cleanup(name):
    name = name.replace(";","")
    name = name.strip()
    name = name.capitalize()
    return name
'''
  
# code snippet whose execution time is to be measured 
stmt_pandas_str = ''' 
messy_names.str.replace(";","").str.strip().str.capitalize()
'''

stmt_pandas_apply = '''
messy_names.apply(cleanup)
'''

stmt_pandas_vectorize='''
pd.Series(np.vectorize(cleanup)(messy_names))
'''

In [38]:
timeit.timeit(setup = setup, 
                    stmt = stmt_pandas_str, 
                    number = 10000) 

1.6925081669999145

In [39]:
timeit.timeit(setup = setup, 
                    stmt = stmt_pandas_apply, 
                    number = 10000) 

0.5813515840000036

In [40]:
timeit.timeit(setup = setup, 
                    stmt = stmt_pandas_vectorize, 
                    number = 10000) 

0.39205824999999095

.str() yöntemleri son derece uygun olsa da, performans söz konusu olduğunda np.vectorize() kullanmayı unutma!

# SON!